# Análisis de Rankings Musicales con la API de Spotify

En este proyecto se utiliza la API de Spotify para construir un dataset musical a partir de playlists oficiales, con el objetivo de analizar rankings de canciones por país y explorar métricas de popularidad y características musicales.

In [5]:
# Importación de Librerias

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Autenticación con la API de Spotify

Para acceder a los datos públicos de Spotify se utiliza el flujo de autenticación **Client Credentials**.  
Este método permite consultar información pública (playlists, canciones y artistas) sin necesidad de acceder a datos personales de usuarios.

Cada persona que desee ejecutar este notebook debe crear su propia aplicación en Spotify for Developers y obtener sus credenciales.

#### Paso a paso para crear las credenciales

1. Ingresá a: https://developer.spotify.com/
2. Iniciá sesión con tu cuenta de Spotify _(gratuita o premium)_.
3. Aceptá los términos y condiciones _(solo la primera vez)_.
4. Abrí el menú superior y seleccioná **Dashboard**.
5. Hacé clic en el botón **Create App** / **Crear aplicación**.
6. Completá el formulario con los siguientes datos:
   
   - **App name:**  
     `Spotify Top 40 Analysis` (o cualquier nombre descriptivo)
   
   - **App description:**  
     Proyecto académico para análisis de rankings musicales
   
   - **Redirect URI:**  
     Podés ingresar cualquier valor, por ejemplo:  
     `http://localhost`  
     _(Este campo no se utiliza en el proyecto, pero Spotify lo requiere obligatoriamente)._

7. Confirmá la creación de la aplicación.

Una vez creada, Spotify te proporcionará dos credenciales:
- **Client ID**
- **Client Secret**

Estas credenciales se utilizan para solicitar un **access token temporal**, el cual es necesario para realizar solicitudes a la API de Spotify.  
Por motivos de seguridad, estas claves **no deben compartirse ni subirse a repositorios públicos**.


In [12]:
# --------------------------------
# SOLICITUD DE CREDENCIALES
## En este bloque se solicita al usuario que ingrese sus credenciales personales de Spotify.
## Esto permite ejecutar el notebook con distintas cuentas sin modificar el código fuente, manteniendo buenas prácticas de seguridad.
# --------------------------------

def request_credentials():
    """
    Solicita al usuario sus credenciales de Spotify y valida que no estén vacías.
    """
    CLIENT_ID = input("Ingresá tu Client ID de Spotify: ").strip()
    CLIENT_SECRET = input("Ingresá tu Client Secret de Spotify: ").strip()

    if not CLIENT_ID or not CLIENT_SECRET:
        raise ValueError("Las credenciales no pueden estar vacías.")

    return CLIENT_ID, CLIENT_SECRET


# ---------------------------------------------------
# AUTENTIFICACIÓN
# Vaildación de credenciales en Spotify
# ---------------------------------------------------

def get_spotify_token(client_id: str, client_secret: str) -> str:
    """
    Obtiene un access token válido desde la API de Spotify usando Client Credentials.
    """
    auth_url = "https://accounts.spotify.com/api/token"

    response = requests.post(
        auth_url,
        data={"grant_type": "client_credentials"},
        auth=(client_id, client_secret)
    )

    if response.status_code != 200:
        raise ConnectionError(
            f"Error al autenticar con Spotify ({response.status_code}): {response.text}"
        )

    data = response.json()

    if "access_token" not in data:
        raise KeyError("La respuesta no contiene un access token.")

    return data["access_token"]


# ---------------------------------------------------
# VALIDACIÓN
# Vaildación de errores en las credenciales
# ---------------------------------------------------

try:
    CLIENT_ID, CLIENT_SECRET = request_credentials()
    token = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
    headers = {"Authorization": f"Bearer {token}"}
    print("Autenticación exitosa.")
except ValueError as ve:
    print("Error en las credenciales:", ve)
except ConnectionError as ce:
    print("Error de conexión con Spotify:", ce)
except Exception as e:
    print("Error inesperado:", e)

# ---------------------------------------------------
# AUTENTICACIÓN DE API
# Token temporal obtenido correctamente
# Una vez obtenida la autenticación, se construye el encabezado (`headers`) que será utilizado en todas las solicitudes posteriores a la API de Spotify.
# ---------------------------------------------------

# headers queda disponible para las siguientes solicitudes
headers = {
    "Authorization": f"Bearer {token}"
}


KeyboardInterrupt: Interrupted by user

In [7]:
# --------------------------------
# PLAY LIST
# Extracción de playlist en Spotify
# --------------------------------

playlists = {
    "Argentina": "ID_PLAYLIST_ARG",
    "Global": "ID_PLAYLIST_GLOBAL"
}

# --------------------------------
# TRACKS
# Extracción de Traks en Spotify
# --------------------------------

tracks_data = []

for country, playlist_id in playlists.items():
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    response = requests.get(url, headers=headers)
    items = response.json()["items"]

    for item in items:
        track = item["track"]
        tracks_data.append({
            "track_name": track["name"],
            "artist": track["artists"][0]["name"],
            "country": country,
            "popularity": track["popularity"],
            "duration_ms": track["duration_ms"],
            "explicit": track["explicit"],
            "track_id": track["id"]
        })


NameError: name 'playlists' is not defined

In [ ]:
# --------------------------------
# VISUALIZACIÓN
# Información extraida de Spotify
# --------------------------------

df = pd.DataFrame(tracks_data)
df.head()
df.info()
df.describe()

In [ ]:
# --------------------------------
# GRÁFICAS
# Visualización de las consultas
# --------------------------------

sns.barplot(
    data=df,
    x="country",
    y="popularity"
)
plt.title("Popularidad promedio por país")
plt.show()

sns.histplot(df["popularity"], bins=20)
plt.title("Distribución de popularidad")
plt.show()

In [ ]:
df.to_csv("data/spotify_top_tracks.csv", index=False)